In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [2]:
MODEL_ID = "facebook/nllb-200-distilled-600M"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model     = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID).to(device)


pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [6]:
# FLORES‑200 language codes for our 5 targets
target_langs = {
    "Hindi"   : "hin_Deva",
    "Spanish" : "spa_Latn",
    "French"  : "fra_Latn",
    "German"  : "deu_Latn",
    "Chinese" : "zho_Hans"   # Simplified Chinese
}

SRC_CODE = "eng_Latn"       # English source code

def translate(text: str) -> dict:
    tokenizer.src_lang = SRC_CODE
    encoded = tokenizer(text, return_tensors="pt").to(device)

    out = {}
    for lang, tgt_code in target_langs.items():
        generated = model.generate(
            **encoded,
            forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_code),
            max_length=256
        )
        out[lang] = tokenizer.decode(generated[0], skip_special_tokens=True)
    return out


In [7]:
sentence = input("Enter an English sentence: ").strip()
for lang, translation in translate(sentence).items():
    print(f"{lang:>8}: {translation}")


c:\Users\soham\.conda\envs\myenv\lib\site-packages\transformers\models\m2m_100\modeling_m2m_100.py:552: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


   Hindi: मेरा नाम सोहम है
 Spanish: Mi nombre es Soham
  French: Je m'appelle Soham.
  German: Mein Name ist Soham .
 Chinese: 我叫索罕.
